In [2]:
pip install pycaret

  Using cached pycaret-3.1.0-py3-none-any.whl (483 kB)
  Using cached numpy-1.23.5-cp310-cp310-macosx_11_0_arm64.whl (13.4 MB)
  Using cached pandas-1.5.3-cp310-cp310-macosx_11_0_arm64.whl (10.9 MB)
  Using cached scipy-1.10.1-cp310-cp310-macosx_12_0_arm64.whl (28.8 MB)
  Using cached joblib-1.3.2-py3-none-any.whl (302 kB)
  Using cached scikit_learn-1.2.2-cp310-cp310-macosx_12_0_arm64.whl (8.5 MB)
  Using cached pyod-1.1.0-py3-none-any.whl
  Using cached imbalanced_learn-0.11.0-py3-none-any.whl (235 kB)
  Using cached category_encoders-2.6.2-py2.py3-none-any.whl (81 kB)
  Using cached lightgbm-4.1.0.tar.gz (1.7 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached numba-0.58.1-cp310-cp310-macosx_11_0_arm64.whl (2.6 MB)
  Using cached importlib_metadata-6.8.0-py3-none-any.whl (22 kB)
  Using cached cloudpickle-3.0.0-py3-none-any.whl (20 kB)
  Us

In [1]:
# check installed version (must be >3.0)
import pycaret
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from yellowbrick.regressor import ResidualsPlot
import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns

pycaret.__version__

ModuleNotFoundError: No module named 'pycaret'

In [4]:
data = pd.read_csv('dataset-all_exp.csv')
data

,seeds,hcl,Asc-acid,silver,temp,loss
0,24.0,850,215,195,27,12.45
1,24.0,712,167,440,27,10.19
2,24.0,269,288,335,27,11.11
3,24.0,281,484,380,27,11.00
4,24.0,488,391,126,27,8.67
...,...,...,...,...,...,...
107,24.0,80,100,120,27,6.34
108,24.0,100,100,120,27,7.39
109,24.0,450,100,120,27,9.73
110,24.0,160,100,120,27,10.19


In [9]:
data_loss_lessthan_8 = data[data['loss'] < 8]  
data_loss_lessthan_8

,seeds,hcl,Asc-acid,silver,temp,loss
9,24.0,323,350,100,27,6.66
10,24.0,286,342,100,27,6.74
19,38.0,166,416,128,37,7.69
20,40.0,203,435,117,36,6.96
21,39.0,232,446,127,35,7.81
22,40.0,234,444,106,35,6.98
23,34.0,208,476,108,35,6.94
24,25.0,443,485,108,38,7.57
25,34.0,193,492,122,36,7.18
26,33.0,234,478,104,35,6.94


In [80]:
# import pycaret regression and init setup
from pycaret.regression import *

s = setup(data, target = 'loss', session_id = 321, normalize = True) # 70%  of data goes in training and 30% in testing

,Description,Value
0,Session id,321
1,Target,loss
2,Target type,Regression
3,Original data shape,"(112, 6)"
4,Transformed data shape,"(112, 6)"
5,Transformed train set shape,"(78, 6)"
6,Transformed test set shape,"(34, 6)"
7,Numeric features,5
8,Preprocess,True
9,Imputation type,simple


In [81]:
# import RegressionExperiment and init the class
from pycaret.regression import RegressionExperiment
exp = RegressionExperiment()

In [82]:
# compare baseline models
best = compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,0.6082,0.7891,0.8236,0.6925,0.0722,0.0603,0.7200
ada,AdaBoost Regressor,0.7377,0.9287,0.9153,0.6100,0.0829,0.0752,0.8120
gbr,Gradient Boosting Regressor,0.6863,1.0708,0.9374,0.5875,0.0837,0.0681,0.7250
rf,Random Forest Regressor,0.6996,1.2047,0.9937,0.5537,0.0894,0.0700,0.7490
knn,K Neighbors Regressor,0.7442,1.5767,1.1179,0.4595,0.1018,0.0746,0.7250
xgboost,Extreme Gradient Boosting,0.7875,1.4778,1.1653,0.3517,0.1038,0.0780,0.7290
lightgbm,Light Gradient Boosting Machine,0.9447,1.6630,1.1935,0.2789,0.1107,0.0968,0.9300
br,Bayesian Ridge,1.0701,1.8518,1.3087,0.1976,0.1223,0.1122,0.7330
ridge,Ridge Regression,1.0740,1.8617,1.3103,0.1749,0.1219,0.1119,0.7300
lar,Least Angle Regression,1.0764,1.8702,1.3129,0.1671,0.1220,0.1120,0.7120


Processing:   0%|          | 0/81 [00:00<?, ?it/s]

In [83]:
rf = create_model('rf')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.4272,0.2386,0.4885,0.8349,0.0442,0.0446
1,0.8898,2.1194,1.4558,0.5743,0.1158,0.0806
2,0.3557,0.2016,0.4490,0.7593,0.0445,0.0388
3,0.3561,0.3507,0.5922,0.7811,0.0510,0.0344
4,0.3295,0.2298,0.4793,0.7793,0.0428,0.0326
5,0.5525,0.6270,0.7918,0.7880,0.0726,0.0588
6,1.1425,2.8890,1.6997,-0.0568,0.1457,0.0993
7,1.0458,2.0548,1.4335,0.5985,0.1340,0.1085
8,0.7472,1.1209,1.0587,0.3790,0.1117,0.0942


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [84]:
tuned_rf = tune_model(rf)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.4338,0.2535,0.5035,0.8246,0.0512,0.0484
1,1.1780,2.7787,1.6669,0.4419,0.1425,0.1153
2,0.4512,0.3819,0.6180,0.5439,0.0587,0.0471
3,0.5379,0.6098,0.7809,0.6194,0.0683,0.0518
4,0.4212,0.3771,0.6141,0.6377,0.0564,0.0425
5,0.6765,0.7840,0.8854,0.7349,0.0809,0.0720
6,1.2247,2.9622,1.7211,-0.0836,0.1475,0.1060
7,1.3965,3.1191,1.7661,0.3905,0.1721,0.1556
8,0.7340,1.2032,1.0969,0.3334,0.1180,0.0955


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [99]:
plot_model(tuned_rf, 'residuals', save = True) #Fig_S7

'Residuals.png'

In [101]:
# interpret summary model
interpret_model(rf, plot = 'summary', save = True)

In [90]:
evaluate_model(tuned_rf)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [91]:
# predict on test set
holdout_pred = predict_model(tuned_rf)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Random Forest Regressor,0.3878,0.3157,0.5619,0.8595,0.0600,0.0456


In [92]:
# show predictions df
holdout_pred.head()

,seeds,hcl,Asc-acid,silver,temp,loss,prediction_label
102,17.0,911,351,362,37,9.88,9.9027
22,40.0,234,444,106,35,6.98,7.2607
8,24.0,1000,335,500,27,10.28,10.6771
89,50.0,752,421,330,35,9.90,9.9617
6,24.0,330,322,159,27,10.91,10.9342


In [94]:
# save pipeline
save_model(best, 'loss_pred')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=FastMemory(location=C:\Users\Anish\AppData\Local\Temp\joblib),
          steps=[('numerical_imputer',
                  TransformerWrapper(include=['seeds', 'hcl', 'Asc-acid',
                                              'silver', 'temp'],
                                     transformer=SimpleImputer())),
                 ('categorical_imputer',
                  TransformerWrapper(include=[],
                                     transformer=SimpleImputer(strategy='most_frequent'))),
                 ('normalize', TransformerWrapper(transformer=StandardScaler())),
                 ('clean_column_names',
                  TransformerWrapper(transformer=CleanColumnNames())),
                 ('trained_model',
                  ExtraTreesRegressor(n_jobs=-1, random_state=321))]),
 'loss_pred.pkl')

In [96]:
# load pipeline
loaded_best_pipeline = load_model('loss_pred')
loaded_best_pipeline

Transformation Pipeline and Model Successfully Loaded


Pipeline(memory=FastMemory(location=C:\Users\Anish\AppData\Local\Temp\joblib),
         steps=[('numerical_imputer',
                 TransformerWrapper(include=['seeds', 'hcl', 'Asc-acid',
                                             'silver', 'temp'],
                                    transformer=SimpleImputer())),
                ('categorical_imputer',
                 TransformerWrapper(include=[],
                                    transformer=SimpleImputer(strategy='most_frequent'))),
                ('normalize', TransformerWrapper(transformer=StandardScaler())),
                ('clean_column_names',
                 TransformerWrapper(transformer=CleanColumnNames())),
                ('trained_model',
                 ExtraTreesRegressor(n_jobs=-1, random_state=321))])